In [1]:
from NLP_Learning import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import BertTokenizerFast
import pandas as pd

/home/oskar/NLP/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data = read_csv()

In [3]:
# Try with ISO-8859-1
try:
    data = pd.read_csv('/home/oskar/NLP/datasets/Corona_NLP_train.csv', encoding='ISO-8859-1')
    print("File read successfully with ISO-8859-1 encoding.")
except UnicodeDecodeError as e:
    print(f"Failed to read file with ISO-8859-1: {e}")

# If the above fails, try with Windows-1252
try:
    data = pd.read_csv('/home/oskar/NLP/datasets/Corona_NLP_train.csv', encoding='cp1252')
    print("File read successfully with Windows-1252 encoding.")
except UnicodeDecodeError as e:
    print(f"Failed to read file with Windows-1252: {e}")

data['Target'] = data['Sentiment'].replace({value:key for key, value in enumerate(data['Sentiment'].unique())})

File read successfully with ISO-8859-1 encoding.
Failed to read file with Windows-1252: 'charmap' codec can't decode byte 0x81 in position 153284: character maps to <undefined>


/tmp/ipykernel_143432/1995724205.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Target'] = data['Sentiment'].replace({value:key for key, value in enumerate(data['Sentiment'].unique())})


In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Parameters for the model
input_size = 520  # The number of features in the input (e.g., size of the embedding vector)
hidden_size = 100  # The number of features in the hidden state of the LSTM
output_size = 5  # The size of the output, e.g., 1 for a regression task

model = SimpleLSTMModel(input_size, hidden_size, output_size, 10).to(device)
dataset = TextClassificationDataset(data['OriginalTweet'].values, data['Target'].values, tokenizer)

train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True, drop_last=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

gradient_norms = {}

# Функция для инициализации ключей в словаре
def init_gradient_norms(model):
    for name, parameter in model.named_parameters():
        if parameter.requires_grad:
            gradient_norms[name] = []

# Функция для сохранения норм градиентов после каждого обратного распространения
def save_gradients(model):
    for name, parameter in model.named_parameters():
        if parameter.grad is not None:
            gradient_norms[name].append(parameter.grad.norm().item())

init_gradient_norms(model)

In [6]:
for epoch in range(10):
    # Forward pass
    total = 0
    correct = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(torch.float32)
        attention_mask = batch['masks']
        labels = batch['labels']

        input_ids, masks, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids).squeeze()
        loss = criterion(outputs, labels)
        
        # Backward pass и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        save_gradients(model)  # Печать нормы градиентов
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(accuracy)

0.27334143377885783


In [ ]:
sdfgdsgfsd

In [ ]:
import time

for epoch in range(1):
    # Start timing the epoch
    epoch_start = time.time()

    # Initialize counters
    total = 0
    correct = 0

    for batch in train_dataloader:
        batch_start = time.time()

        # Loading data to device
        input_ids = batch['input_ids'].to(torch.float32)
        attention_mask = batch['masks']
        labels = batch['labels']

        data_load_time = time.time() - batch_start
        print(f"Data Loading Time: {data_load_time:.4f} sec")
        

In [ ]:
import time

for epoch in range(1):
    # Start timing the epoch
    epoch_start = time.time()

    # Initialize counters
    total = 0
    correct = 0

    for batch in train_dataloader:
        batch_start = time.time()

        # Loading data to device
        input_ids = batch['input_ids'].to(torch.float32)
        attention_mask = batch['masks']
        labels = batch['labels']

        data_load_time = time.time() - batch_start

        # Transfer data to the device (GPU or CPU)
        transfer_to_device_start = time.time()
        input_ids, masks, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        transfer_to_device_time = time.time() - transfer_to_device_start

        # Forward pass
        forward_start = time.time()
        outputs = model(input_ids).squeeze()
        loss = criterion(outputs, labels)
        forward_time = time.time() - forward_start

        # Backward pass and optimization
        backward_start = time.time()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        backward_time = time.time() - backward_start

        # Save gradients
        save_gradient_start = time.time()
        save_gradients(model)  # Save or print gradient norms
        save_gradient_time = time.time() - save_gradient_start

        # Calculation of metrics
        metric_calc_start = time.time()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        metric_calc_time = time.time() - metric_calc_start

        # Print time taken for each step
        print(f"Data Loading Time: {data_load_time:.4f} sec")
        print(f"Transfer to Device Time: {transfer_to_device_time:.4f} sec")
        print(f"Forward Pass Time: {forward_time:.4f} sec")
        print(f"Backward Pass Time: {backward_time:.4f} sec")
        print(f"Save Gradient Time: {save_gradient_time:.4f} sec")
        print(f"Metric Calculation Time: {metric_calc_time:.4f} sec")
        print(f"Correct Predictions Count: {correct}")

    accuracy = correct / total
    epoch_time = time.time() - epoch_start
    print(f"Epoch Accuracy: {accuracy:.4f}")
    print(f"Total Epoch Time: {epoch_time:.4f} sec")
